### Imports

In [43]:
import warnings
warnings.filterwarnings("ignore")

!pwd
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/DSCI565_HW/Project/')
!pwd

/content/drive/MyDrive/DSCI565_HW/Project
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/DSCI565_HW/Project


In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [45]:
# !pip install yfinance
import yfinance as yf

In [46]:
symbol = 'GOOGL'  # change as needed
start_date = '2009-01-01'
end_date = '2023-11-18'
df = yf.download(symbol, start=start_date, end=end_date)
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-02,7.722723,8.053554,7.645145,8.041041,8.041041,144275580
2009-01-05,8.033033,8.289289,7.882883,8.209459,8.209459,195364440
2009-01-06,8.332833,8.528529,8.167918,8.359860,8.359860,256750992
2009-01-07,8.216216,8.281031,7.976727,8.058308,8.058308,179600220
2009-01-08,7.964965,8.137888,7.941441,8.137888,8.137888,143883972
...,...,...,...,...,...,...
2023-11-13,131.779999,132.589996,131.250000,132.089996,132.089996,18324800
2023-11-14,134.190002,135.699997,133.320007,133.619995,133.619995,32395200
2023-11-15,134.869995,135.029999,133.570007,134.619995,134.619995,23861500


In [47]:
# Signal shifting - shifting up or down the volume time series

# Function to augment dataset by shifting signals
def augment_data_ss(stock_data, shift_steps):
    augmented_data_posshift = stock_data.copy()
    augmented_data_negshift = stock_data.copy()

    # Positive and Negative shift
    augmented_data_posshift['Volume'] = augmented_data_posshift['Volume'].shift(-shift_steps)
    augmented_data_negshift['Volume'] = augmented_data_negshift['Volume'].shift(shift_steps)

    # Drop rows with NaN values introduced by shifting
    augmented_data_posshift = augmented_data_posshift.dropna()
    augmented_data_negshift = augmented_data_negshift.dropna()

    return augmented_data_posshift, augmented_data_negshift


def augment_rowwise_ss(original_data, augmented_data):
    # Concatenate the original and augmented dataframes row-wise
    augmented_rowwise = pd.concat([original_data, augmented_data], axis=0)

    # Drop rows with NaN values (introduced by shifting in the original data)
    augmented_rowwise = augmented_rowwise.dropna()
    return augmented_rowwise


shift_steps = 10 #(range 5-10)
stock_data = df.copy()
augmented_signalshifting_data = pd.DataFrame()

# Augment data with signal shifting
for i in range(5, shift_steps+1): # starting with 5 shift steps
  shiftsteps = i

  augmented_data_posshift, augmented_data_negshift = augment_data_ss(stock_data, shiftsteps)

  # if i==5:
  #   augmented_signalshifting_data_temp = augment_rowwise_ss(stock_data, augmented_data_posshift)
  #   augmented_signalshifting_data_temp = augment_rowwise_ss(augmented_signalshifting_data_temp, augmented_data_negshift)

  augmented_signalshifting_data_temp = augment_rowwise_ss(augmented_data_posshift, augmented_data_negshift)

  print(augmented_signalshifting_data_temp.shape)
  augmented_signalshifting_data = pd.concat([augmented_signalshifting_data, augmented_signalshifting_data_temp], axis=0)


print(augmented_data_posshift.shape, augmented_data_negshift.shape, stock_data.shape)
augmented_signalshifting_data

(7482, 6)
(7480, 6)
(7478, 6)
(7476, 6)
(7474, 6)
(7472, 6)
(3736, 6) (3736, 6) (3746, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-02,7.722723,8.053554,7.645145,8.041041,8.041041,173446380.0
2009-01-05,8.033033,8.289289,7.882883,8.209459,8.209459,132039828.0
2009-01-06,8.332833,8.528529,8.167918,8.359860,8.359860,177122700.0
2009-01-07,8.216216,8.281031,7.976727,8.058308,8.058308,218497284.0
2009-01-08,7.964965,8.137888,7.941441,8.137888,8.137888,237142620.0
...,...,...,...,...,...,...
2023-11-13,131.779999,132.589996,131.250000,132.089996,132.089996,28940100.0
2023-11-14,134.190002,135.699997,133.320007,133.619995,133.619995,26292300.0
2023-11-15,134.869995,135.029999,133.570007,134.619995,134.619995,30082400.0


In [48]:
augmented_signalshifting_data[augmented_signalshifting_data.isna().any(axis=1)]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [49]:
# Signal filtering

# Function to augment dataset by applying a moving average filter
def augment_data_sf(stock_data, window_size):
    augmented_data = stock_data.copy()

    # Apply a simple moving average filter to each column
    for col in augmented_data.columns:
        if col!='Open':
          augmented_data[col] = augmented_data[col].rolling(window=window_size).mean()

    augmented_data['Open'] = augmented_data['Close'].shift(1)
    augmented_data['Open'][0] = stock_data.copy()['Open'][0]

    for col in stock_data.copy().columns:
        if col!='Open':
          augmented_data[col] = stock_data.copy()[col]


    # Drop rows with NaN values introduced by the moving average filter
    augmented_data = augmented_data.dropna()

    return augmented_data


def augment_rowwise_sf(original_data, augmented_data):
    # Concatenate the original and augmented dataframes row-wise
    augmented_rowwise = pd.concat([original_data, augmented_data], axis=0)

    # Drop rows with NaN values (introduced by the moving average filter in the original data)
    augmented_rowwise = augmented_rowwise.dropna()
    return augmented_rowwise


window_size = [5,10,15,20]  # Adjust window size for moving average filter
stock_data = df.copy()
augmented_signalfiltering_data = pd.DataFrame()

# Augment data with signal filtering
for i in window_size:
  windowsize = i

  # Augment data with signal filtering (moving average)
  augmented_data = augment_data_sf(stock_data, windowsize)


  #if i==window_size[0]:
   #augmented_signalfiltering_data = augment_rowwise_sf(stock_data, augmented_data)

  augmented_signalfiltering_data = augment_rowwise_sf(augmented_signalfiltering_data, augmented_data)
  print(augmented_signalfiltering_data.shape)


augmented_signalfiltering_data

(3742, 6)
(7479, 6)
(11211, 6)
(14938, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-02,7.722723,8.053554,7.645145,8.041041,8.041041,144275580
2009-01-09,8.161311,8.195696,7.842843,7.884635,7.884635,173446380
2009-01-12,8.130030,7.981732,7.763514,7.825075,7.825075,132039828
2009-01-13,8.053153,8.023023,7.767518,7.865866,7.865866,177122700
2009-01-14,7.954354,7.852853,7.451201,7.531782,7.531782,218497284
...,...,...,...,...,...,...
2023-11-13,131.148501,132.589996,131.250000,132.089996,132.089996,18324800
2023-11-14,130.798000,135.699997,133.320007,133.619995,133.619995,32395200
2023-11-15,130.493000,135.029999,133.570007,134.619995,134.619995,23861500


In [50]:
augmented_signalfiltering_data[augmented_signalfiltering_data.isna().any(axis=1)]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [51]:
# Noise Addition

# Function to augment dataset by noise
def noise_addition(stock_data, stdev):

    augmented_data = stock_data.copy()

    # Generation of artificial gaussian random noises
    price = augmented_data['Close']
    volume = augmented_data['Volume']
    priceNoise = np.random.normal(0, stdev*(price/100))
    volumeNoise = np.random.normal(0, stdev*(volume/100))

    # Addition of the artificial noise generated
    augmented_data['Close'] *= (1 + priceNoise/100)
    augmented_data['Low'] *= (1 + priceNoise/100)
    augmented_data['High'] *= (1 + priceNoise/100)
    augmented_data['Adj Close'] *= (1 + priceNoise/100)
    augmented_data['Volume'] *= (1 + volumeNoise/100)
    augmented_data['Open'] = augmented_data['Close'].shift(1)


    # Drop rows with NaN values
    augmented_data = augmented_data.dropna()

    return augmented_data


def augment_noise_addition(original_data, augmented_data):
    # Concatenate the original and augmented dataframes row-wise
    augmented_rowwise = pd.concat([original_data, augmented_data], axis=0)

    # Drop rows with NaN values
    augmented_rowwise = augmented_rowwise.dropna()
    return augmented_rowwise


stdn = [1]  # Adjust standard deviation
stock_data = df.copy()
augmented_noise_addition_data = pd.DataFrame()


for i in stdn:
  std_dev = i

  # Augment data with noise
  augmented_data = noise_addition(stock_data, std_dev)


  # if i==stdn[0]:
  #   augmented_noise_addition_data = augment_noise_addition(stock_data, augmented_data)

  augmented_noise_addition_data = augment_noise_addition(augmented_noise_addition_data, augmented_data)
  print(augmented_noise_addition_data.shape)


augmented_noise_addition_data

(3745, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-05,8.035388,8.297881,7.891053,8.217968,8.217968,7.817393e+11
2009-01-06,8.217968,8.533310,8.172497,8.364546,8.364546,3.937806e+12
2009-01-07,8.364546,8.289124,7.984523,8.066183,8.066183,-4.689818e+11
2009-01-08,8.066183,8.152312,7.955517,8.152312,8.152312,1.034861e+12
2009-01-09,8.152312,8.188860,7.836302,7.878059,7.878059,-4.073802e+12
...,...,...,...,...,...,...
2023-11-13,134.978160,129.793889,128.482151,129.304433,129.304433,-3.928085e+10
2023-11-14,129.304433,136.899527,134.498499,134.801138,134.801138,3.511013e+10
2023-11-15,134.801138,136.856899,135.377155,136.441349,136.441349,-7.244772e+10


In [52]:
augmented_noise_addition_data[augmented_noise_addition_data.isna().any(axis=1)]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [53]:
# Combining all data
print(df.shape, augmented_signalshifting_data.shape, augmented_signalfiltering_data.shape, augmented_noise_addition_data.shape)
df = pd.concat([df, augmented_signalshifting_data, augmented_signalfiltering_data, augmented_noise_addition_data], axis=0)
df

(3746, 6) (44862, 6) (14938, 6) (3745, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-02,7.722723,8.053554,7.645145,8.041041,8.041041,1.442756e+08
2009-01-05,8.033033,8.289289,7.882883,8.209459,8.209459,1.953644e+08
2009-01-06,8.332833,8.528529,8.167918,8.359860,8.359860,2.567510e+08
2009-01-07,8.216216,8.281031,7.976727,8.058308,8.058308,1.796002e+08
2009-01-08,7.964965,8.137888,7.941441,8.137888,8.137888,1.438840e+08
...,...,...,...,...,...,...
2023-11-13,134.978160,129.793889,128.482151,129.304433,129.304433,-3.928085e+10
2023-11-14,129.304433,136.899527,134.498499,134.801138,134.801138,3.511013e+10
2023-11-15,134.801138,136.856899,135.377155,136.441349,136.441349,-7.244772e+10
